# Section 6: From Scikit-Learn to NimbusML


## Goals:
* Learn to write scripts with NimbusML components
* Learn to boost your existing Scikit Learn scripts with NimbusML components

## Why to use NimbusML ?
* Used ML.NET before?
* Used Scikit-Learn before?

### Would you want this?
<img align="middle" src="https://nimbusml.blob.core.windows.net/mlads/goals.png" height=700 />


### NimbusML:
<img align="middle" src="https://nimbusml.blob.core.windows.net/mlads/speed.png" width=550 height=550 />


## Lets start ... 
## Check and if needed install NimbusML package (might take 1 to 7 mins)

In [ ]:
import sys
!{sys.executable.replace('python', 'pip') + ' install --upgrade nimbusml'}
print("wheel installed.")

In [ ]:
# check version
import nimbusml
print (nimbusml.__version__)
# 1.0.0

# copy files to faster disk storage
!mkdir /home/nbuser/local
!cp -R Part3 /home/nbuser/local
print("copy done.")

# run quick tests
cmd = sys.executable + ' -m nimbusml.examples.FastLinearClassifier'
!{cmd}
print("tests ok.")

### Now lets do all the imports:

In [ ]:
# Cell 1

# imports
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier

# NimbusML imports
from nimbusml import Pipeline as NimbusPipeline, FileDataStream
from nimbusml.linear_model import FastLinearBinaryClassifier
from nimbusml.feature_extraction.text import NGramFeaturizer

### Set up train and test data:

In [ ]:
# Cell 2

np.random.seed(0)

# Prepare train and test data
# Twitter sentiment prediction
# Subset of Kaggle Twitter positive/negative sentiment prediction https://www.kaggle.com/c/twitter-analysis  

train_file = 'data/train.tsv'
test_file = 'data/test.tsv'
data_train = pd.read_csv(train_file, header=0, sep='\t', encoding='latin-1') 
data_test = pd.read_csv(test_file, header=0, sep='\t', encoding='latin-1')
print(data_train[:10])

label_column = 'Sentiment'
feature_column = 'SentimentText'
train_X = data_train[feature_column].values.astype('U')
train_y = data_train[label_column]

k = 5000 # cut file into 500 lines
test_X = data_test[feature_column][:k].values.astype('U')
test_y = data_test[label_column][:k]

### Scikit script:

In [ ]:
# Cell 3

# Define pipeline, add transforms and classifier
pipe = Pipeline([
('vect', CountVectorizer()),
('tfidf', TfidfTransformer()),
('clf', SGDClassifier(max_iter=10))])

# Train pipeline
pipe.fit(train_X, train_y)

# Get predictions
test_pred = pipe.predict(test_X)

print(test_pred[:10])
print("acc: %s" % accuracy_score(test_pred, test_y))

### Replace with NimbusML learner:

In [ ]:
# Cell 4

# Define pipeline, add transforms and classifier
pipe = Pipeline([
('vect', CountVectorizer()),
('tfidf', TfidfTransformer()),
('clf', FastLinearBinaryClassifier())])

# Train pipeline
pipe.fit(train_X, train_y)

# Get predictions
test_pred = pipe.predict(test_X)

#print(test_pred[:10])
print("acc: %s" % accuracy_score(test_pred, test_y))

### If you need to look up FastLinearBinaryClassifier details, here is extensive doc site:
https://docs.microsoft.com/en-us/NimbusML
Additional TLC support alias: tlcsupp@microsoft.com


### High level architecture

* Memory passed in by ref
* Memory passed back by copy

<img align="middle" src="https://nimbusml.blob.core.windows.net/mlads/architecture.png" width=600 heigth=400 />



### Optimized NimbusML script:

In [ ]:
# Cell 5

schema = 'sep=tab col=Label:R4:0 col=SentimentText:TX:1 header=+'
trainDs = FileDataStream(train_file, schema)
testDs = FileDataStream(test_file, schema)

pipe = NimbusPipeline([
  NGramFeaturizer() << {'Features':'SentimentText'},
  FastLinearBinaryClassifier()])

# Train pipeline
pipe.fit(trainDs)

# Get predictions
test_pred = pipe.predict(testDs)

#print(test_pred[:10])
print("acc: %s" % accuracy_score(test_pred['PredictedLabel'][:k], test_y))

## Recap:
* Created simple scikit learn script
* Used NimbusML learner in scikit learn pipeline
* Used NimbusML transformers and learner in NimbusML pipeline 

And if you would have run a whole dataset:
<img align="middle" src="https://nimbusml.blob.core.windows.net/mlads/scikit2nimbusml.png"/>

### Resources
- [Scikit Learn](https://scikit-learn.org/stable/)
-[Training of Python scikit-learn and deep learning models on Azure](https://docs.microsoft.com/en-us/azure/architecture/reference-architectures/ai/training-python-models)
- [NimbusML Overview](https://docs.microsoft.com/en-us/nimbusml/overview)
- [A Quick Start of NimbusML - Classification](https://docs.microsoft.com/en-us/nimbusml/tutorials/a_a-classification-with-synthetic-data)
- [A Quick Start of NimbusML - Regression](https://docs.microsoft.com/en-us/nimbusml/tutorials/a_b-regression-with-synthetic-data)
-[Sentiment Analysis 3 - Combining NimbusML and Scikit-learn](https://docs.microsoft.com/en-us/nimbusml/tutorials/b_c-sentiment-analysis-3-combining-nimbusml-and-scikit-learn)
- [Loading, Saving and Serving Models with NimbusML](https://docs.microsoft.com/en-us/nimbusml/loadsavemodels)